# Collaborative Filtering Recommender System

In general, Collaborative filtering (CF) is more commonly used than content-based systems because it usually gives better results and is relatively easy to understand (from an overall implementation perspective). The algorithm has the ability to do feature learning on its own, which means that it can start to learn for itself what features to use. 

CF can be divided into **Memory-Based Collaborative Filtering** and **Model-Based Collaborative filtering**. 

In this tutorial, we will implement Model-Based CF by using singular value decomposition (SVD) and Memory-Based CF by computing cosine similarity. 



In [54]:
#输出多个output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'
#Open hinterland
%config Completer.use_jedi = False

In [55]:
import numpy as np
import pandas as pd

## Data

In [56]:
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=column_names)

In [57]:
df.head()

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


In [58]:
movie_titles = pd.read_csv("Movie_Id_Titles")
movie_titles.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [59]:
df = pd.merge(df,movie_titles,on='item_id')
df

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)
2,79,50,4,891271545,Star Wars (1977)
3,2,50,5,888552084,Star Wars (1977)
4,8,50,5,879362124,Star Wars (1977)
...,...,...,...,...,...
99998,840,1674,4,891211682,Mamma Roma (1962)
99999,655,1640,3,888474646,"Eighth Day, The (1996)"
100000,655,1637,3,888984255,Girls Town (1996)
100001,655,1630,3,887428735,"Silence of the Palace, The (Saimt el Qusur) (1..."


In [60]:
n_users = df.user_id.nunique()
n_items = df.item_id.nunique()

print('Num. of Users:',n_users)
print('Num of Movies:',n_items)

Num. of Users: 944
Num of Movies: 1682


Recommendation Systems by their very nature are very difficult to evaluate, but we will still show you how to evaluate them in this tutorial. In order to do this, we'll split our data into two sets. However, we won't do our classic X_train,X_test,y_train,y_test split. Instead we can actually just segement the data into two sets of data:

In [61]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.25,random_state=1113)

In [62]:
train_data.shape

(75002, 5)

## Memory-Based Collaborative Filtering
### Construct System

Memory-Based Collaborative Filtering approaches can be divided into two main sections: **user-item filtering** and **item-item filtering**. 

A *user-item filtering* will take a particular user, find users that are similar to that user based on similarity of ratings, and recommend items that those similar users liked. 

In contrast, *item-item filtering* will take an item, find users who liked that item, and find other items that those users or similar users also liked. It takes items and outputs other items as recommendations. 

* *Item-Item Collaborative Filtering*: “Users who liked this item also liked …”
* *User-Item Collaborative Filtering*: “Users who are similar to you also liked …”

In both cases, you create a user-item matrix. Since we have split the data into testing and training we will need to create two ``[944 x 1682]`` matrices (all users by all movies). 

---

After you have built the user-item matrix you calculate the similarity and create a similarity matrix. 

* For *Item-Item Collaborative Filtering* the similarity values between items are measured by observing all the users who have rated both items. 

* For *User-Item Collaborative Filtering* the similarity values between users are measured by observing all the items that are rated by both users.

---

A distance metric commonly used in recommender systems is *cosine similarity*, where the ratings are seen as vectors in ``n``-dimensional space and the similarity is calculated based on the angle between these vectors. 
Cosine similiarity for users *a* and *m* can be calculated using the formula below, where you take dot product of  the user vector *$u_k$* and the user vector *$u_a$* and divide it by multiplication of the Euclidean lengths of the vectors.
<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?s_u^{cos}(u_k,u_a)=\frac{u_k&space;\cdot&space;u_a&space;}{&space;\left&space;\|&space;u_k&space;\right&space;\|&space;\left&space;\|&space;u_a&space;\right&space;\|&space;}&space;=\frac{\sum&space;x_{k,m}x_{a,m}}{\sqrt{\sum&space;x_{k,m}^2\sum&space;x_{a,m}^2}}"/>

To calculate similarity between items *m* and *b* you use the formula:

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?s_u^{cos}(i_m,i_b)=\frac{i_m&space;\cdot&space;i_b&space;}{&space;\left&space;\|&space;i_m&space;\right&space;\|&space;\left&space;\|&space;i_b&space;\right&space;\|&space;}&space;=\frac{\sum&space;x_{a,m}x_{a,b}}{\sqrt{\sum&space;x_{a,m}^2\sum&space;x_{a,b}^2}}
"/>

Your first step will be to create the user-item matrix. Since you have both testing and training data you need to create two matrices. 

In [63]:
#New Tricks
for line in train_data.itertuples():
    print(line)
    break

Pandas(Index=16543, user_id=893, item_id=294, rating=3, timestamp=874827789, title='Liar Liar (1997)')


In [64]:
#Create two user-item matrices, one for training and another for testing
train_data_matrix = np.empty((n_users, n_items))
train_data_matrix[:] = np.nan
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]

test_data_matrix = np.empty((n_users, n_items))
test_data_matrix[:] = np.nan
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

In [65]:
train_data_matrix.shape
test_data_matrix.shape

(944, 1682)

(944, 1682)

**We should do mean normalization within each row**

The idea here is that some users may tend always to give high or low ratings to all movies. The relative difference in the ratings that these users give is more important than the absolute values. To give an example: suppose, user *k* gives 4 stars to his favourite movies and 3 stars to all other good movies. Suppose now that another user *t* rates movies that he/she likes with 5 stars, and the movies he/she fell asleep over with 3 stars. These two users could have a very similar taste but treat the rating system differently.

In [66]:
train_data_matrix

array([[ 5.,  3.,  4., ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [67]:
train_data_row_mean = np.nanmean(train_data_matrix,axis=1,keepdims=True)
train_data_row_mean.shape

(944, 1)

In [68]:
train_data_matrix = train_data_matrix - train_data_row_mean

这里我们不对test_data_matrix做mean normalization, 在做预测的时候，把mean加回去即可

之后，我们将所有的NaN替换为0

In [69]:
train_data_matrix[np.isnan(train_data_matrix)] = 0
test_data_matrix[np.isnan(test_data_matrix)] = 0

In [70]:
train_data_matrix

array([[ 1.46700508, -0.53299492,  0.46700508, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

You can use the pairwise_distances function from sklearn to calculate the cosine similarity. Note, the output will range from 0 to 1 since the ratings are all positive.

In [71]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric="cosine")
item_similarity = pairwise_distances(train_data_matrix.T, metric="cosine")

In [72]:
train_data_matrix.shape
user_similarity.shape
item_similarity.shape

(944, 1682)

(944, 944)

(1682, 1682)

Next step is to make predictions. You have already created similarity matrices: `user_similarity` and `item_similarity` and therefore you can make a prediction by applying following formula for user-based CF:

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?\hat{x}_{k,m}&space;=&space;\bar{x}_{k}&space;&plus;&space;\frac{\sum\limits_{u_a}&space;sim_u(u_k,&space;u_a)&space;(x_{a,m}&space;-&space;\bar{x_{u_a}})}{\sum\limits_{u_a}|sim_u(u_k,&space;u_a)|}"/>

We have do the mean normalization before, no more need to do here

When making a prediction for item-based CF you don't need to correct for users average rating since query user itself is used to do predictions.

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?\hat{x}_{k,m}&space;=&space;\frac{\sum\limits_{i_b}&space;sim_i(i_m,&space;i_b)&space;(x_{k,b})&space;}{\sum\limits_{i_b}|sim_i(i_m,&space;i_b)|}"/>

In [73]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        pred = similarity.dot(ratings) / np.sum(np.abs(similarity), axis=1, keepdims=True) + train_data_row_mean
    elif type == 'item':
        pred = ratings.dot(similarity) / np.sum(np.abs(similarity), axis=1) + train_data_row_mean
    return pred

In [74]:
user_prediction = predict(train_data_matrix, user_similarity, type='user')
item_prediction = predict(train_data_matrix, item_similarity, type='item')

In [75]:
user_prediction
item_prediction

array([[3.61517803, 3.50859547, 3.49510108, ..., 3.53173589, 3.53299492,
        3.53299492],
       [3.7321575 , 3.6184294 , 3.60709024, ..., 3.64313599, 3.64444444,
        3.64444444],
       [2.9576909 , 2.84376721, 2.83091969, ..., 2.86821191, 2.86956522,
        2.86956522],
       ...,
       [4.30798268, 4.19637786, 4.18441749, ..., 4.22099304, 4.22222222,
        4.22222222],
       [3.47040886, 3.3592326 , 3.34830718, ..., 3.38333841, 3.38461538,
        3.38461538],
       [3.75326667, 3.64046312, 3.62886009, ..., 3.66539381, 3.66666667,
        3.66666667]])

array([[3.52722606, 3.53684779, 3.53468566, ..., 3.53323178, 3.53299492,
        3.53299492],
       [3.64410161, 3.64477848, 3.64497537, ..., 3.64431515, 3.64444444,
        3.64444444],
       [2.86957673, 2.86979867, 2.86943462, ..., 2.8691606 , 2.86956522,
        2.86956522],
       ...,
       [4.22162022, 4.22241293, 4.22256792, ..., 4.22248708, 4.22222222,
        4.22222222],
       [3.38228779, 3.38574113, 3.38723682, ..., 3.38461538, 3.38461538,
        3.38461538],
       [3.66648991, 3.66673582, 3.66681465, ..., 3.66666667, 3.66666667,
        3.66666667]])

### Evaluation
There are many evaluation metrics but one of the most popular metric used to evaluate accuracy of predicted ratings is *Root Mean Squared Error (RMSE)*. 
<img src="https://latex.codecogs.com/gif.latex?RMSE&space;=\sqrt{\frac{1}{N}&space;\sum&space;(x_i&space;-\hat{x_i})^2}" title="RMSE =\sqrt{\frac{1}{N} \sum (x_i -\hat{x_i})^2}" />

Since you only want to consider predicted ratings that are in the test dataset, you filter out all other elements in the prediction matrix with `prediction[ground_truth.nonzero()]`.

In [76]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [77]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 1.0215457056441115
Item-based CF RMSE: 1.039878435708755


### How to recommend

Now we have two results from Item-Item and User-Item collaborative filtering, let's give some recommendation


In [78]:
moviemat = df.pivot_table(index='user_id',columns='item_id',values='rating')
moviemat

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


我们要给user_id = 0的人进行推荐5个电影，首先使用User-Item collaborative filtering来推荐

In [79]:
df[df["user_id"]==0]

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
584,0,172,5,881250949,"Empire Strikes Back, The (1980)"
952,0,133,1,881250949,Gone with the Wind (1939)


0号用户很喜欢星球大战一类的冒险动作片

In [80]:
#因为可能推荐的电影用户0看过，所以我们选出前10的推荐电影
user_prediction[0].argsort()[-10:][::-1]

array([ 49, 173, 126,  99,  97, 317, 312,  63, 482, 171])

In [81]:
#前10电影的item_id
user_0_item_id = user_prediction[0].argsort()[-10:][::-1] + 1
user_0_item_id

array([ 50, 174, 127, 100,  98, 318, 313,  64, 483, 172])

In [82]:
#去除掉看过的电影
df[df["user_id"]==0]["item_id"]
diff = set(user_0_item_id) - set(df[df["user_id"]==0]["item_id"])
[item for item in user_0_item_id if item in diff]
user_item_results = [item for item in user_0_item_id if item in diff]

0       50
584    172
952    133
Name: item_id, dtype: int64

[174, 127, 100, 98, 318, 313, 64, 483]

In [83]:
for i in user_item_results:
    movie = df[df["item_id"]==i]["title"].unique()
    print(movie)

['Raiders of the Lost Ark (1981)']
['Godfather, The (1972)']
['Fargo (1996)']
['Silence of the Lambs, The (1991)']
["Schindler's List (1993)"]
['Titanic (1997)']
['Shawshank Redemption, The (1994)']
['Casablanca (1942)']


***Users who are similar to you also like "Raiders of the Lost Ark", "Godfather", "Fargo".....***

可以看出推荐的都是和星球大战为同一类型的影片

再用Item-Item的方法推荐一下影片:

In [84]:
item_0_item_id = item_prediction[0].argsort()[-10:][::-1] + 1
diff = set(item_0_item_id) - set(df[df["user_id"]==0]["item_id"])
item_item_results = [item for item in item_0_item_id if item in diff]
for i in item_item_results:
    movie = df[df["item_id"]==i]["title"].unique()
    print(movie)

['Theodore Rex (1995)']
['Nadja (1994)']
['Turbo: A Power Rangers Movie (1997)']
['D3: The Mighty Ducks (1996)']
['Free Willy 2: The Adventure Home (1995)']
['Flipper (1996)']
['Operation Dumbo Drop (1995)']
['Batman Forever (1995)']
['Kull the Conqueror (1997)']
['Batman Returns (1992)']


***Users who liked this item also liked "Theodore Rex", "Nadja"....***

从效果上来看并不是很好的推荐，可能是应为该用户对电影的rating很少，导致结果并不准确

---

每当加进来新的用户，就把matrix更新一下重新拟合模型，然后根据最新的模型给出推荐，如果新加入的用户没有对任何电影做rating，那么将无法实现推荐

---

Memory-based algorithms are easy to implement and produce reasonable prediction quality. 
The drawback of memory-based CF is that it doesn't scale to real-world scenarios and doesn't address the well-known cold-start problem, that is when new user or new item enters the system. Model-based CF methods are scalable and can deal with higher sparsity level than memory-based models, but also suffer when new users or items that don't have any ratings enter the system.

## Model-Based Collaborative Filtering

Model-based Collaborative Filtering is based on **matrix factorization (MF)** which has received greater exposure, mainly as an unsupervised learning method for latent variable decomposition and dimensionality reduction. Matrix factorization is widely used for recommender systems where it can deal better with scalability and sparsity than Memory-based CF. The goal of MF is to learn the latent preferences of users and the latent attributes of items from known ratings (learn features that describe the characteristics of ratings) to then predict the unknown ratings through the dot product of the latent features of users and items.(和吴恩达提到的方法思路一致)

When you have a very sparse matrix, with a lot of dimensions, by doing matrix factorization you can restructure the  user-item matrix into low-rank structure, and you can represent the matrix by the multiplication of two low-rank matrices, where the rows contain the latent vector. You fit this matrix to approximate your original matrix, as closely as possible, by multiplying the low-rank matrices together, which fills in the entries missing in the original matrix.

Let's calculate the sparsity level of MovieLens dataset:

In [85]:
sparsity=round(1.0-len(df)/float(n_users*n_items),3)
print('The sparsity level of MovieLens100K is ' +  str(sparsity*100) + '%')

The sparsity level of MovieLens100K is 93.7%


To give an example of the learned latent preferences of the users and items: let's say for the MovieLens dataset you have the following information: _(user id, age, location, gender, movie id, director, actor, language, year, rating)_. By applying matrix factorization the model learns that important user features are _age group (under 10, 10-18, 18-30, 30-90)_, _location_ and _gender_, and for movie features it learns that _decade_, _director_ and _actor_ are most important. Now if you look into the information you have stored, there is no such feature as the _decade_, but the model can learn on its own. The important aspect is that the CF model only uses data (user_id, movie_id, rating) to learn the latent features. If there is little data available model-based CF model will predict poorly, since it will be more difficult to learn the latent features. 

Models that use both ratings and content features are called **Hybrid Recommender Systems** where both Collaborative Filtering and Content-based Models are combined. Hybrid recommender systems usually show higher accuracy than Collaborative Filtering or Content-based Models on their own: they are capable to address the cold-start problem better since if you don't have any ratings for a user or an item you could use the metadata from the user or item to make a prediction. 

### SVD

A well-known matrix factorization method is **Singular value decomposition (SVD)**. Collaborative Filtering can be formulated by approximating a matrix `X` by using singular value decomposition. The winning team at the Netflix Prize competition used SVD matrix factorization models to produce product recommendations, for more information I recommend to read articles: [Netflix Recommendations: Beyond the 5 stars](http://techblog.netflix.com/2012/04/netflix-recommendations-beyond-5-stars.html) and [Netflix Prize and SVD](http://buzzard.ups.edu/courses/2014spring/420projects/math420-UPS-spring-2014-gower-netflix-SVD.pdf).
The general equation can be expressed as follows:
<img src="https://latex.codecogs.com/gif.latex?X=USV^T" title="X=USV^T" />


Given `m x n` matrix `X`:
* *`U`* is an *`(m x r)`* orthogonal matrix
* *`S`* is an *`(r x r)`* diagonal matrix with non-negative real numbers on the diagonal
* *V^T* is an *`(r x n)`* orthogonal matrix

Elements on the diagnoal in `S` are known as *singular values of `X`*. 


Matrix *`X`* can be factorized to *`U`*, *`S`* and *`V`*. The *`U`* matrix represents the feature vectors corresponding to the users in the hidden feature space and the *`V`* matrix represents the feature vectors corresponding to the items in the hidden feature space.


Then you can make a prediction by taking dot product of *`U`*, *`S`* and *`V^T`*.

这个方法和吴恩达的区别在于，吴恩达 only learn the latent features of the movies, 而SVD方法会学习user和movie的feature.

In [86]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

#Get SVD components from train matrix. Choose k
u, s, vt = svds(train_data_matrix, k=20)
s_diag_matrix = np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt) + train_data_row_mean

In [87]:
X_pred
X_pred.shape

array([[4.07877653, 3.6176229 , 3.4347678 , ..., 3.53942189, 3.53299492,
        3.53299492],
       [3.91351977, 3.63834105, 3.51985329, ..., 3.6402315 , 3.64444444,
        3.64444444],
       [2.79662335, 2.83213578, 2.84867429, ..., 2.88089511, 2.86956522,
        2.86956522],
       ...,
       [4.26522008, 4.21010365, 4.26755048, ..., 4.22417162, 4.22222222,
        4.22222222],
       [3.93332032, 3.11645104, 3.16984426, ..., 3.39270767, 3.38461538,
        3.38461538],
       [3.74206219, 3.65048066, 3.6653656 , ..., 3.66790903, 3.66666667,
        3.66666667]])

(944, 1682)

**Evaluation**:

In [88]:
print('Model-Based CF RMSE: ' + str(rmse(X_pred, test_data_matrix)))

Model-Based CF RMSE: 0.9881610373528238


The RMSE is lower than the Memory-Based CF. However, carelessly addressing only the relatively few known entries is highly prone to overfitting. SVD can be very slow and computationally expensive. More recent work minimizes the squared error by applying alternating least square or stochastic gradient descent and uses regularization terms to prevent overfitting.

Then, we also give the suggestions for the top 10 movies to user 0:

In [89]:
df[df["user_id"]==0]

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
584,0,172,5,881250949,"Empire Strikes Back, The (1980)"
952,0,133,1,881250949,Gone with the Wind (1939)


In [90]:
Model_0_item_id = X_pred[0].argsort()[-10:][::-1] + 1
diff = set(Model_0_item_id) - set(df[df["user_id"]==0]["item_id"])
Model_item_results = [item for item in Model_0_item_id if item in diff]
for i in Model_item_results:
    movie = df[df["item_id"]==i]["title"].unique()
    print(movie)

['Fargo (1996)']
['Raiders of the Lost Ark (1981)']
['Princess Bride, The (1987)']
['Return of the Jedi (1983)']
['Alien (1979)']
['Sling Blade (1996)']
['Swingers (1996)']
['Usual Suspects, The (1995)']
['Hunt for Red October, The (1990)']


***There are the movie recommendations for you: "Fargo", "Raiders of the Lost Ark"....***

可以看出推荐的影片都是以冒险为主，效果很不错

## Review

We did a great job:
* We have covered how to implement simple **Collaborative Filtering** methods, both memory-based CF and model-based CF.
* **Memory-based models** are based on similarity between items or users, where we use cosine-similarity.
* **Model-based CF** is based on matrix factorization where we use SVD to factorize the matrix.
* Building recommender systems that perform well in cold-start scenarios (where little data is available on new users and items) remains a challenge. The standard collaborative filtering method performs poorly is such settings.

## More References

If you want to tackle your own recommendation system analysis, check out these data sets. Note: The files are quite large in most cases, not all the links may stay up to host the data, but the majority of them still work. Or just Google for your own data set!

**Movies Recommendation:**

MovieLens - Movie Recommendation Data Sets http://www.grouplens.org/node/73

Yahoo! - Movie, Music, and Images Ratings Data Sets http://webscope.sandbox.yahoo.com/catalog.php?datatype=r

Jester - Movie Ratings Data Sets (Collaborative Filtering Dataset) http://www.ieor.berkeley.edu/~goldberg/jester-data/

Cornell University - Movie-review data for use in sentiment-analysis experiments http://www.cs.cornell.edu/people/pabo/movie-review-data/

**Music Recommendation:**

Last.fm - Music Recommendation Data Sets http://www.dtic.upf.edu/~ocelma/MusicRecommendationDataset/index.html

Yahoo! - Movie, Music, and Images Ratings Data Sets http://webscope.sandbox.yahoo.com/catalog.php?datatype=r

Audioscrobbler - Music Recommendation Data Sets http://www-etud.iro.umontreal.ca/~bergstrj/audioscrobbler_data.html

Amazon - Audio CD recommendations http://131.193.40.52/data/

**Books Recommendation:**

Institut für Informatik, Universität Freiburg - Book Ratings Data Sets http://www.informatik.uni-freiburg.de/~cziegler/BX/
Food Recommendation:

Chicago Entree - Food Ratings Data Sets http://archive.ics.uci.edu/ml/datasets/Entree+Chicago+Recommendation+Data
Merchandise Recommendation:

**Healthcare Recommendation:**

Nursing Home - Provider Ratings Data Set http://data.medicare.gov/dataset/Nursing-Home-Compare-Provider-Ratings/mufm-vy8d

Hospital Ratings - Survey of Patients Hospital Experiences http://data.medicare.gov/dataset/Survey-of-Patients-Hospital-Experiences-HCAHPS-/rj76-22dk

**Dating Recommendation:**

www.libimseti.cz - Dating website recommendation (collaborative filtering) http://www.occamslab.com/petricek/data/
Scholarly Paper Recommendation:

National University of Singapore - Scholarly Paper Recommendation http://www.comp.nus.edu.sg/~sugiyama/SchPaperRecData.html